In [2]:
%pip install tensorflow scikit-learn pandas numpy


  Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
  Using cached scikit_learn-1.4.0-1-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata (5.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.10.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.2.0-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.14.1-cp311-cp311-win_amd64.whl.metadata (6.9 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp311-cp311-win_amd64.whl (1.5 MB)
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ ----

In [5]:
import pandas as pd
#https://www.kaggle.com/code/teowaihong/exercise-pose-classification-with-pytorch/notebook
# Load the dataset
file_path = '../train.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe
df.head()

,pose_id,pose,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,...,z_left_heel,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index
0,0,squats_up,-0.382815,-48.231250,-54.405792,0.137189,-50.040543,-51.997875,0.502047,-50.058890,...,33.283375,-12.302963,52.377000,21.567219,16.609533,54.351162,15.890617,-16.779694,55.382572,2.050140
1,1,situp_down,54.146880,-12.822491,5.564175,56.762527,-11.221117,-0.363063,56.795986,-10.608183,...,-8.073100,-37.519974,4.045297,62.104233,-60.468560,6.281939,-8.185631,-52.589350,-3.852214,59.750070
2,2,situp_down,9.891440,-54.147266,85.344970,12.784414,-55.229970,88.534775,14.006874,-54.291880,...,-132.024460,-61.214890,53.040688,-16.786700,-5.844672,99.829030,-166.543060,-73.826830,38.470097,-40.519830
3,3,jumping_jacks_up,0.904673,-51.350130,-33.606970,1.338871,-53.172337,-30.013737,1.743913,-53.050697,...,51.615970,-15.543400,38.251835,34.033176,16.610210,46.572014,38.680614,-18.784650,45.098710,20.343110
4,4,jumping_jacks_down,-3.153129,-55.255062,-17.745928,-2.046205,-57.477790,-18.198952,-1.506304,-57.428230,...,-2.849554,2.811829,56.000500,56.605590,0.566345,66.231050,-19.252045,-3.451297,60.029484,45.503677


In [1]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Suppress TensorFlow warnings (optional, not recommended)
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load the dataset
df = pd.read_csv('train.csv')  # Make sure the path is correct

# Extract features and labels
X = df.drop(['pose_id', 'pose'], axis=1)
y = df['pose']

# Encode the labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# One-hot encode the labels
y_categorical = to_categorical(y_encoded)

# Normalize the feature data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)

# Reshape input data for LSTM. LSTM expects data in the shape of [samples, time steps, features].
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define the model architecture
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))  # The output layer with a softmax activation

# Compile the model with the Adam optimizer
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.3)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy*100:.2f}%')

# Save the model in the native Keras format
model.save('exercise_recognition_model2.keras')

C:\Users\JR\AppData\Local\Temp\ipykernel_26708\97613008.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd




Epoch 1/100


10/10 [==============================] - 8s 239ms/step - loss: 2.2823 - accuracy: 0.1892 - val_loss: 2.2632 - val_accuracy: 0.3788
Epoch 2/100
10/10 [==============================] - 0s 15ms/step - loss: 2.2371 - accuracy: 0.3197 - val_loss: 2.2210 - val_accuracy: 0.4432
Epoch 3/100
10/10 [==============================] - 0s 9ms/step - loss: 2.1947 - accuracy: 0.3899 - val_loss: 2.1718 - val_accuracy: 0.4697
Epoch 4/100
10/10 [==============================] - 0s 8ms/step - loss: 2.1355 - accuracy: 0.4372 - val_loss: 2.1131 - val_accuracy: 0.4848
Epoch 5/100
10/10 [==============================] - 0s 12ms/step - loss: 2.0765 - accuracy: 0.4568 - val_loss: 2.0417 - val_accuracy: 0.4962
Epoch 6/100
10/10 [==============================] - 0s 8ms/step - loss: 2.0086 - accuracy: 0.4454 - val_loss: 1.9587 - val_accuracy: 0.4924
Epoch 7/100
10/10 [==============================] - 0s 9ms/step - loss: 1.9228 - accuracy: 0.4682 - val_loss: 1.8708 - val_accuracy: 0.4924
Epoch

In [2]:
import joblib

# Save the LabelEncoder to a file after fitting
joblib.dump(label_encoder, 'label_encoder.joblib')

['label_encoder.joblib']

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Assuming X_train, X_test, y_train, y_test are already defined and preprocessed from your dataset
# Load the dataset
df = pd.read_csv('train.csv')  # Make sure the path is correct

# Extract features and labels
X = df.drop(['pose_id', 'pose'], axis=1)
y = df['pose']

# Encode the labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# One-hot encode the labels
y_categorical = to_categorical(y_encoded)

# Normalize the feature data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)

# Reshape input data for LSTM. LSTM expects data in the shape of [samples, time steps, features].
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define a slightly modified model architecture
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))  # Adjusted number of units
model.add(Dropout(0.4))  # Slightly adjusted dropout rate
model.add(LSTM(25))  # Adjusted number of units in the second LSTM layer
model.add(Dropout(0.3))  # Adjusted dropout rate
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)  # Adjusted epochs and batch size

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy*100:.2f}%')


Epoch 1/30
22/22 [==============================] - 8s 72ms/step - loss: 2.2724 - accuracy: 0.2282 - val_loss: 2.2378 - val_accuracy: 0.4830
Epoch 2/30
22/22 [==============================] - 0s 6ms/step - loss: 2.1901 - accuracy: 0.4579 - val_loss: 2.1575 - val_accuracy: 0.5114
Epoch 3/30
22/22 [==============================] - 0s 8ms/step - loss: 2.0884 - accuracy: 0.4822 - val_loss: 2.0377 - val_accuracy: 0.5341
Epoch 4/30
22/22 [==============================] - 0s 6ms/step - loss: 1.9323 - accuracy: 0.4893 - val_loss: 1.8818 - val_accuracy: 0.5057
Epoch 5/30
22/22 [==============================] - 0s 6ms/step - loss: 1.7725 - accuracy: 0.5036 - val_loss: 1.7146 - val_accuracy: 0.4886
Epoch 6/30
22/22 [==============================] - 0s 5ms/step - loss: 1.5984 - accuracy: 0.5193 - val_loss: 1.5584 - val_accuracy: 0.5057
Epoch 7/30
22/22 [==============================] - 0s 6ms/step - loss: 1.4497 - accuracy: 0.5421 - val_loss: 1.4293 - val_accuracy: 0.5455
Epoch 8/30
22/22 [=